Kollar de första raderna så jag ser att datan laddats korrekt:

In [1]:
import pandas as pd

df = pd.read_csv("housing.csv")
df.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Skriver ut en lista med alla variabler (och funderar på vilken jag vill ha som Y och vilken/vilka att jämföra med)

In [12]:
print(list(df.columns))

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']


Kollar vilka typer av värden som finns i variablerna  
(numeriska: int, float och/eller kategoriska: object?)

Det gör jag för att identifiera kategoriska variabler (som jag inte vill ha med i min linjära regression). 

In [3]:
df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

"ocean_proximity" = kategorisk variabel med dtype: object

statisktisk sammanställning för att få en överblick av variablernas olika värden: 

In [4]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


Kollar efter saknade värden (bra att veta om någon variabel saknar data, pga olika mängd värden vid ex jämförelse)

In [5]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Kolumnen total_bedrooms har 207 rader med saknade värden.

Separerar numeriska och kategoriska kolumner (vid linjär regression måste datan vara numerisk, därför är det bra att se vilka variabler som har kategoriska värden som behöver korrigeras om om jag vill använda dessa som X (features) sedan): 

In [11]:
numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = df.select_dtypes(include=["object"]).columns

print(f"numerical_cols: {list(numerical_cols)}, categorical_cols: {list(categorical_cols)}")




numerical_cols: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value'], categorical_cols: ['ocean_proximity']


Efter att ha jämfört de olika variablerna har jag valt att sätta min målvariabel (Y) till median_house_value med anlidningen att den är:  
- numerisk och kontinuerlig
- rimlig att jämföra andra variabler med(då den förmodligen påverkas av exempelvis läge och inkomst). 
  
Den gör:
- regressionskoefficienter tolkbara
- signifikans meningsfull
- resultat “realistiska”

Kollar vilka värden som finns i ocean_proximity för att sedan kunna göra om dem till numeriska värden: 

In [ ]:
df["ocean_proximity"].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

Kollar också antalet värden per kategori för att få syn på vilka som är vanligast och om det finns väldigt få av några (viktigt vid tolkning sedan): 

In [14]:
df["ocean_proximity"].value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

För att ännu tydligare se propotionerna (hur vanligt förekommande i kategorin):

In [15]:
df["ocean_proximity"].value_counts(normalize=True)

ocean_proximity
<1H OCEAN     0.442636
INLAND        0.317393
NEAR OCEAN    0.128779
NEAR BAY      0.110950
ISLAND        0.000242
Name: proportion, dtype: float64

ISLAND - få av detta värde. Vid interpretation av regressionskoeffficienter måste man ha detta vid åtanke, då det kan leda till stora standardfel och "konstiga" koefficienter. 

One-hot encoding av ocean_proximity (för att göra den numerisk): 